# Filter the `Hellisotherpeople/DebateSum` dataset for hateful content / extremist content

The `Hellisotherpeople/DebateSum` has a lot of radical & extremist content, like pro-marxist/communist advocacy and racist-activism

In [2]:
%pip install pystemmer datasets xgboost transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.0/303.0 kB 1.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 37.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 72.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 31.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 81.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 27.8 MB/s eta 0:00:00
  Created wheel for pystemmer: filename=PyStemmer-2.2.0.1-cp310-cp310-linux_x86_64.whl size=579734 sha256=317eee7e9a82470103486609a08bd51be8ae008b6f6336796409fd278f0b1ac5
  Stored in directory: /root/.cache/pip/wheels/45/7d/2c/a7ebb8319

In [3]:
import torch
from torch.cuda import is_available
assert is_available()

In [5]:
data = load_dataset("Hellisotherpeople/DebateSum",split='train',streaming=False)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [6]:
#foo =load_dataset('Hellisotherpeople/DebateSum',split='train',streaming=True)

from datasets import load_dataset
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import string
import Stemmer
stemmer = Stemmer.Stemmer("english")
import numpy as np
from sklearn.metrics.pairwise import cosine_distances
from scipy.stats import scoreatpercentile
import warnings
warnings.filterwarnings("ignore")

def is_extremist(d):
    x = d['Full-Document']
    if 'LatCrit' in x or 'critical consciousness' in x.lower():
       return True
    if (
        'imperialist' in x or 'imperialism' in x.lower() or 'anti-colonial' in x.lower()
    ) and (
        'anti-capitalis' in x or 'discourse' in x or 'antiracist' in x or 'postmodern' in x or 'social justice' in x.lower()
    ):
       return True
    if 'Karl Marx' in x or 'poststructuralist' in x or 'Marist' in x or 'Marxism' in x or 'Trotsky' in x or 'Karl Marx' in x:
       return True
    if 'queer' in x or 'LGBTQ' in x or 'LGBT' in x or 'heteronormative' in x.lower() or 'hetero-normative' in x.lower():
       return True
    if 'misogyny' in x.lower() or 'Patriarchy' in x or 'ecofeminist' in x or 'patriarchies' in x or 'feminist' in x.lower() or 'sexism' in x:
       return True
    if 'straight white male' in x or 'white, heterosexual, and European men' in x.lower() or 'Western man' in x or 'homophobic' in x or 'xenophobic' in x:
       return True
    if (
        'indigenous movement' in x.lower() or 'indigenous activ' in x.lower() or ' land back' in x or 'decolonize' in x.lower() or 'BIPOC' in x.lower() or 'BLM' in x or
        'white suprem' in x.lower() or 'eurocentric' in x.lower() or 'rapper' in x or 'bitch' in x or 'hip-hop' in x or 'super predators' in x or 'crack babies' in x
    ):
       return True
    if 'trans-movement' in x or 'trans-identity' in x:
       return True
    if 'critical theory' in x or 'praxis' in x:
       return True
    return False

# get streaming-object for extremist text
#l = load_dataset("Hellisotherpeople/DebateSum", split='train',streaming=True).filter(is_extremist)

# get candidates for extremist text
#ltexts = [e for e in l]

ltexts = [e for e in data.filter(is_extremist)]


Filter:   0%|          | 0/240566 [00:00<?, ? examples/s]

In [7]:
print(f"{len(ltexts)} possible extremist texts")

16074 possible extremist texts


In [9]:
ltexts[-1000]

{'Unnamed: 0': 226928,
 'Full-Document': 'Judicial politics are contradictory like that. Any strategy for responding to the Gorsuch nomination and Trump’s judicial agenda more broadly must begin with an appreciation of these paradoxes, and a clear-eyed view of the Supreme Court’s conservative constitutional role. It is not a matter of opposing Gorsuch because of his originalism — still less of defending the judiciary from capture by conservative jurisprudence. It is a matter of resisting the judiciary itself as a reactionary institution. It is about focusing on the Court, not on Trump’s pick. A movement of mass opposition and disruption must target institutions rather than individuals, power rather than personalities. A filibuster by Senate Democrats would certainly be a welcome (and newfound) display of parliamentary intransigence. But it would be a mistake to stake all our hopes on the success of a campaign of protest aimed at holding the Democrats to account. There are few reasons t

In [ ]:

corpus = [d['Full-Document'] for d in ltexts] + [d['Extract'] for d in ltexts]
corpus_names = [d['OriginalDebateFileName'] for d in ltexts] + [d['OriginalDebateFileName'] for d in ltexts]

# Define a function to preprocess a single text
def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()
    # Remove URLs using a regular expression
    text = re.sub(r'http\S+', '', text)
    # Remove special characters and punctuation
    text = ''.join([char if (char not in ['-',"'", '"', '‘', '’', '/']) else " " for char in text])
    text = ''.join([char for char in text if char not in string.punctuation])
    # Split the text into words
    words = text.split()
    # Remove stopwords (you can replace this list with your own)
    stopwords = set([
        "i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your", "yours", "yourself", "yourselves", "he", "him", "his", "himself", "she", "her", "hers", "herself","it", "its", "itself", "they", "them", "their", "theirs", "themselves", "what", "which", "who", "whom", "this", "that", "these", "those", "am", "is", "are", "was", "were", "be", "been", "being", "have", "has", "had", "having", "do", "does", "did", "doing", "a", "an", "the", "and", "but", "if", "or", "because", "as", "until", "while", "of", "at", "by", "for", "with", "about", "against", "between", "into", "through", "during", "before", "after", "above", "below", "to", "from", "up", "down", "in", "out", "on", "off", "over", "under", "again", "further", "then", "once", "here", "there", "when", "where", "why", "how", "all", "any", "both", "each", "few", "more", "most", "other", "some", "such", "no", "nor", "not", "only", "own", "same", "so", "than", "too", "very", "s", "t", "can", "will", "just", "don", "should", "now", "d", "ll", "m", "o", "re", "ve", "y", "ain", "aren", "couldn", "didn", "doesn", "hadn", "hasn", "haven", "isn", "ma", "mightn", "mustn", "needn", "shan", "shouldn", "wasn", "weren", "won", "wouldn","also"
    ])
    #words = [word for word in words if word not in stopwords]
    words = [stemmer.stemWord(word) for word in words if word not in stopwords]
    # Join the words back into a single text
    preprocessed_text = ' '.join(words)
    return preprocessed_text

# Apply preprocessing to each text in list_of_texts
preprocessed_texts = [preprocess_text(text) for text in corpus]
# Create a TfidfVectorizer with your preprocessed texts
#vectorizer = TfidfVectorizer(max_features=2000)
#vectorizer_1000 = TfidfVectorizer(max_features=1000)
vectorizer = TfidfVectorizer(max_features=500)
# Fit the vectorizer on the preprocessed texts and transform them into TF-IDF vectors
tfidf_matrix = vectorizer.fit_transform(preprocessed_texts)
# Calculate the global centroid of TF-IDF vectors
centroid = np.mean(tfidf_matrix, axis=0)
# Calculate cosine distances from the centroid to all vectors in the corpus
distances = cosine_distances(np.asarray(centroid), tfidf_matrix)



In [16]:
# Specify the quantiles you want to compute
quantiles_to_compute = [0.05, 0.2, 0.5, 0.6, 0.75, 0.8, 0.9, 0.95, 0.99]
# Calculate the specified quantiles of distances
quantiles = {q:scoreatpercentile(distances, q * 100) for q in quantiles_to_compute}
# filterout those texts that are probably not extremist
quantile_threshold = 0.6

## the positive set for classifying

In [17]:
ltests2_docs = [l for l,d in zip(ltexts, distances.reshape(-1)) if d<quantiles[quantile_threshold]]
ltests2 = [s for s,d in zip(corpus, distances.reshape(-1)) if d<quantiles[quantile_threshold]]
# THIS IS THE POSITIVE SET
print(len(ltests2))

19289


In [19]:
# the negative set
def not_extremist(d):
    if len(str(d['Full-Document']).split(' ')) < 500:
        return False
    return not is_extremist(d)

# get non-extremist content
#notl = load_dataset("Hellisotherpeople/DebateSum", split='train',streaming=True).filter(not_extremist).take(10000) # a subsample that are not extremist texts
notltexts = [e for e in data.filter(not_extremist)][:10000]
#corpus = [d['Full-Document'] for d in notltexts]

In [20]:
print(f"size of negative set:{len(notltexts)}")

size of negative set:10000


In [ ]:
#corpus_names = [d['OriginalDebateFileName'] for d in notltexts]

import torch.nn.functional as F
from torch import Tensor
from transformers import AutoTokenizer, AutoModel
import torch

tokenizer = AutoTokenizer.from_pretrained('intfloat/e5-small-v2')
model = AutoModel.from_pretrained('intfloat/e5-small-v2')
model = model.to('cuda:0')

def average_pool(
    last_hidden_states: Tensor,
    attention_mask: Tensor
) -> Tensor:
    last_hidden = last_hidden_states.masked_fill(~attention_mask[..., None].bool(), 0.0)
    return last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]

In [31]:



def batch_encode_text(texts, batch_size=64):
    # Initialize the tokenizer and model
    # Initialize lists to store the embeddings
    embeddings_all = []
    # Process the texts in batches
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        # Tokenize the batch
        inputs = tokenizer(batch, padding=True, truncation=True, return_tensors="pt", max_length=512)
        # Encode the batch
        for k,v in inputs.items():
            inputs[k] = v.to('cuda:0')
        with torch.no_grad():
            outputs = model(**inputs)

        # Extract the embeddings (e.g., the last hidden states)
        embeddings = average_pool(outputs.last_hidden_state, inputs['attention_mask'])
        embeddings = embeddings.cpu().detach().numpy()
        # Add the batch of embeddings to the list
        embeddings_all.append(embeddings)

    # Concatenate the embeddings from all batches
    concatenated_embeddings = np.concatenate(embeddings_all,axis=0)
    return concatenated_embeddings

# Tokenize the input texts
#batch_dict = tokenizer(["it mystifies a world that cannot actually satisfy people\'s real human needs. But to do this we must be more radical rather than less radical, using whatever re- sources are available to us as teachers and practitioners to expose the myths of liberal as well as conservative world-views. Speaking to law teacher"], max_length=512, padding=True, truncation=True, return_tensors='pt')
#batch_encode_text(["it mystifies a world that cannot actually satisfy people\'s real human needs. But to do this we must be more radical rather than less radical, using whatever re- sources are available to us as teachers and practitioners to expose the myths of liberal as well as conservative world-views. Speaking to law teacher", "cal rather than less radical, using whatever re- sources are available to us as teachers and practitioners to"," cannot actually satisfy people\'s real human needs. But to do this we must be more radical rather tha"], batch_size=2)
#outputs = model(**batch_dict)
#embeddings = average_pool(outputs.last_hidden_state, batch_dict['attention_mask'])

# convert all the texts to vectors
def pipeline_for_classification(texts):
    # tfidf
    tfidf_vectors = vectorizer.fit_transform([
        preprocess_text(text) for text in texts
    ]).toarray()
    # then minilm
    nn_outputs = batch_encode_text(texts, batch_size=64)
    return np.concatenate((tfidf_vectors, nn_outputs), axis=1)


In [32]:
features_notl = pipeline_for_classification([str(d['Full-Document']) for d in notltexts])
# make classification features for the leftist texts
features_l = pipeline_for_classification([str(s) for s in ltests2])
# make targets
targets = [0.0]*features_notl.shape[0] + [1.0]*features_l.shape[0]


In [33]:
import pickle

In [34]:
# save corpus and session for later
with open('extremist-text-classification.pkl','wb') as pcon:
    pickle.dump(features_notl, pcon)
    pickle.dump(features_l, pcon)
    pickle.dump(targets, pcon)
    pickle.dump(notltexts, pcon)
    pickle.dump(ltests2, pcon)
    pickle.dump(tokenizer, pcon)

In [20]:
with open('extremist-text-classification.pkl','rb') as pcon:
    features_notl = pickle.load(pcon)
    features_l = pickle.load(pcon)
    targets = pickle.load(pcon)
    notltexts = pickle.load(pcon)
    ltests2 = pickle.load(pcon)
    tokenizer = pickle.load(pcon)


In [35]:
# XGB classification

import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Assuming you have your features in 'features' and your binary targets in 'targets'

# Split the data into training and holdout sets
X_train, X_holdout, y_train, y_holdout = train_test_split(
    np.concatenate((features_notl, features_l),axis=0),
    targets, test_size=0.1, random_state=42
)

# Convert the data to DMatrix format (required by XGBoost)
dtrain = xgb.DMatrix(X_train, label=y_train)
dholdout = xgb.DMatrix(X_holdout, label=y_holdout)

# Define XGBoost parameters
params = {
    'objective': 'binary:logistic',
    'eval_metric': 'logloss',  # You can choose another metric like 'auc' if desired
    'max_depth': 5,
    'eta': 0.27,
    'gamma': 0.1,
    'min_child_weight': 12.0,
    'subsample': 0.3,
    'colsample_bytree': 0.5,
    'scale_pos_weight': 0.75,
    'seed': 42
}

# Specify the number of boosting rounds and early stopping criteria
num_rounds = 500  # You can adjust this number
early_stopping_rounds = 20

# Train the XGBoost model with early stopping
watchlist = [(dtrain, 'train'), (dholdout, 'eval')]
xgbmod = xgb.train(params, dtrain, num_rounds, evals=watchlist,
                  early_stopping_rounds=early_stopping_rounds, verbose_eval=True)

# Make predictions on the holdout set
y_pred = xgbmod.predict(dholdout, ntree_limit=xgbmod.best_iteration)

# Convert predicted probabilities to binary labels
y_pred_binary = [1 if p >= 0.5 else 0 for p in y_pred]

# Evaluate the model on the holdout set
accuracy = accuracy_score(y_holdout, y_pred_binary)
print(f"Accuracy on Holdout Set: {accuracy}")


[0]	train-logloss:0.52268	eval-logloss:0.52273
[1]	train-logloss:0.41302	eval-logloss:0.41337
[2]	train-logloss:0.32866	eval-logloss:0.32882
[3]	train-logloss:0.27085	eval-logloss:0.27121
[4]	train-logloss:0.22522	eval-logloss:0.22713
[5]	train-logloss:0.18966	eval-logloss:0.19187
[6]	train-logloss:0.16179	eval-logloss:0.16465
[7]	train-logloss:0.13860	eval-logloss:0.14083
[8]	train-logloss:0.11957	eval-logloss:0.12163
[9]	train-logloss:0.10313	eval-logloss:0.10538
[10]	train-logloss:0.09008	eval-logloss:0.09221
[11]	train-logloss:0.07885	eval-logloss:0.08080
[12]	train-logloss:0.07049	eval-logloss:0.07240
[13]	train-logloss:0.06273	eval-logloss:0.06495
[14]	train-logloss:0.05606	eval-logloss:0.05844
[15]	train-logloss:0.05043	eval-logloss:0.05285
[16]	train-logloss:0.04581	eval-logloss:0.04810
[17]	train-logloss:0.04161	eval-logloss:0.04354
[18]	train-logloss:0.03823	eval-logloss:0.04000
[19]	train-logloss:0.03480	eval-logloss:0.03634
[20]	train-logloss:0.03204	eval-logloss:0.03349
[2

In [23]:
def pipeline_for_inference(texts):
    # tfidf
    tfidf_vectors = vectorizer.fit_transform([
        preprocess_text(text) for text in texts
    ]).toarray()
    # then minilm
    nn_outputs = batch_encode_text(texts, batch_size=32)
    # features for xgboosr
    features_inference =np.concatenate((tfidf_vectors, nn_outputs), axis=1)
    X_inference = xgb.DMatrix(features_inference)
    return xgbmod.predict(X_inference)

def pipelines_for_inference_batched(inputs):
    pred = pipeline_for_inference(inputs['Full-Document'])
    return {
        'pred':pred.tolist(),
    }

# stream and make predictions on data


# WARNING: this is too much
# get the full dataset and those which are very high in pred
predset_streaming = load_dataset(
    'Hellisotherpeople/DebateSum',split='train',streaming=True
).filter(
    lambda x : (len(x['Full-Document'].split(' '))>400)
).map(
    #pipelines_for_inference_and_insert_into_example, batched = True, batch_size=32
    pipelines_for_inference_batched, batched = True, batch_size=32*8
).remove_columns(
    ['#CharsAbstract', '#CharsDocument', '#CharsExtract', '#WordsAbstract', '#WordsDocument', '#WordsExtract', 'AbsCompressionRatio', 'Abstract', 'Citation', 'DebateCamp', 'ExtCompressionRatio', 'Extract', 'Tag', 'Unnamed: 0', 'Year']
).take(10)

# takes too long: instead, I should first filter done by keywords: (white suprem
predset = [e for e in predset_streaming]



NameError: ignored

In [22]:
possible_keywords_of_extremism = [s.lower() for s in [
    'latcrit', 'critical consciousness', 'poststructuralist', 'capitalism','capitalist',
    'imperialist', 'imperialism', 'anti-colonial', 'Western man', 'Karl Marx',
    'heteronormative', 'hetero-normative', 'queer', 'LGBTQ', 'LGBT', 'Marist',
    'Marxism', 'Trotsky', 'feminist', 'misogyny', 'Patriarchy', 'ecofeminist',
    'patriarchies', 'sexism', 'straight white male', 'white, heterosexual, and European men',
    'indigenous', 'indigenous', 'land back', 'decolonize', 'BIPOC', 'BLM', 'white suprem', 'eurocentric',
    'rapper', 'bitch', 'hip-hop', 'trans-movement', 'trans-identity', 'critical theory', 'praxis',
    'gender','transgender', 'trans-male', 'trans-female','critical', 'critique','Foucault','postmodern',
    'hegemony', 'hegemonic', 'gay','homosexual','theory', 'revolution', 'movement', 'activist',
    'capitalist','capitalism','bourgeoisie','Bourgeois','sex','class consciousness',
    'lesbian','butch', 'post-modern','toxic masculin','thug',
    'black','people of color', 'reactionary', 'Trump', 'discourse','racial disparities',
    'anti-racist','antiracist','social justice','homophobic','xenophobic','white','black','latino','mexican','global south',
    'anti-poverty','criminal justice',
]]

def is_extremist_candidate(x):
    text = str(x['Full-Document']).lower()
    return any([kw in text for kew in possible_keywords_of_extremism])

SyntaxError: ignored